In [1]:
import pickle
import pandas as pd
import os
import tqdm
os.chdir("D://About_coding/py/car/newCompare")
class Worker:
    def __init__(self, id, x, y):
        self.id = id
        self.x = x
        self.y = y
    
    def __repr__(self) -> str:
        return '({},{})'.format(self.x, self.y)

class Task:
    def __init__(self, id, x, y, price):
        self.id = id
        self.x = x
        self.y = y
        self.price = price
def toSeconds(time)->float:
    h,m,s = str(time).strip().split(":")
    return int(h) * 3600 + int(m) * 60 + float(s)
def count_unstable(match, tasks, workers):
    #原始match: task.id -> worker
    #调整match: task.id -> worker.id
    # match = [w.id for w in match]
    w2t = [-1] * len(match)
    for t in range(len(tasks)):
        w2t[match[t]] = t
    count = 0
    for t in tqdm.tqdm(range(len(tasks))):
        #t和新的w配对判断是否比t和match[t]配对更满意
        for w in range(len(workers)):
            if (dist(tasks[t], workers[match[t]]) > dist(tasks[t], workers[w])) and (tasks[t].price > tasks[w2t[w]].price):
                count += 1
                break
    return count
def dist(worker, task):
    r2 = (worker.x - task.x) ** 2 + (worker.y - task.y) ** 2
    r = pow(r2, 0.5)
    return r
def load_dataset(number):
    workers = []
    tasks = []
    idx = 0
    with open("data/worker_" + str(number) + ".txt", "r") as f:
        while True:
            try:
                line = f.readline().split()
                w = Worker(idx, float(line[0]), float(line[1]))
                workers.append(w)
                idx += 1
            except:
                break
    idx = 0
    with open("data/task_" + str(number) + ".txt", "r") as f:
        while True:
            try:
                line = f.readline().split()
                t = Task(idx, float(line[0]), float(line[1]), float(line[2]))
                tasks.append(t)
                idx += 1
            except:
                break
    return tasks, workers
def toExcel_percent(datain:list,name:str):
    # 创建一个包含数据的DataFrame
    data = {'Column1': datain}
    df = pd.DataFrame(data)

    # 将DataFrame中的值乘以100并转换为百分数格式
    df['Column1'] = df['Column1'].apply(lambda x: '{:.2%}'.format(x))

    # 将DataFrame写入Excel
    writer = pd.ExcelWriter(name, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1', index=False)
    worksheet = writer.sheets['Sheet1']

    # 将Excel中的列宽设置为10
    worksheet.set_column(0, 0, 10)

    # 关闭Excel写入器
    writer.save()

In [4]:
# 计算稳定比例
stable_ratio=[]
for number in [500*i for i in range(1,7)]:
    tasks, workers = load_dataset(number)
    with open("results/alg04_match/alg04_match"+str(number)+".pkl","rb") as fp:
        match=pickle.load(fp)
    cnt = count_unstable(match, tasks, workers)
    stable_ratio.append(1-cnt/number)

toExcel_percent(stable_ratio,"results/alg04 stable ratio.xlsx")


100%|██████████| 3000/3000 [00:07<00:00, 420.95it/s]
